In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.callbacks import get_openai_callback

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

/workspaces/ActiveLoop-Course/env/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [3]:
with get_openai_callback() as cb:
    print(llm_chain("artificial"))
    print(cb)

{'word': 'artificial', 'text': '\n\nSynthetic'}
Tokens Used: 14
	Prompt Tokens: 11
	Completion Tokens: 3
Successful Requests: 1
Total Cost (USD): $0.0


In [4]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'},
 {'text': '\n\nCleverness'},
 {'text': '\n\nandroid'}]

In [5]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nCleverness', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nandroid', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'prompt_tokens': 33, 'total_tokens': 42, 'completion_tokens': 9}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=RunInfo(run_id=UUID('7bd3de61-de4f-438b-8519-5a1b79208f10')))

In [7]:
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")
# or llm_chain.run(word="fan", context="object")

'\n\nSupporter'

In [8]:
llm_chain.predict(word="fan", context="humans")
# or llm_chain.run(word="fan", context="humans")

'\n\nSupporter'

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
words = "words"
template = f"""List all possible {words} as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

List all possible words as substitute for 'artificial' as comma separated.


['synthetic',
 'man-made',
 'fake',
 'imitation',
 'simulated',
 'faux',
 'fabricated',
 'counterfeit',
 'ersatz',
 'false',
 'pretend',
 'mock',
 'spurious',
 'bogus',
 'phony',
 'contrived',
 'manufactured',
 'unnatural',
 'inauthentic']

In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

' Synthetic, simulated, man-made, fake, imitation, fabricated, manufactured, engineered, virtual, computer-generated, digital, non-natural, inauthentic, contrived, ersatz, pseudo, phony, false, unreal, pretend, counterfeit, spurious, bogus, sham, feigned, deceptive, fraudulent, specious, fallacious, misleading, delusive, illusory, fictitious, make-believe, imaginary, unreal, fictive, fanciful, fantastic, mythical, legendary, mythological, fabricated, created, contrived, invented, made-up, unreal, imaginary, fictional, mythical, legendary, mythological, fanciful, fantastic, make-believe, pretend, simulated, virtual, computer-generated, digital, non-natural, inauthentic, ersatz, pseudo, phony, false, counterfeit, spurious, bogus, sham, feigned, deceptive, fraudulent, specious, fallacious, misleading, delusive, illusory, fictitious, make-believe, imaginary, unreal, fictive, fanciful, fantastic, mythical, legendary, mythological, fabricated, created, contrived, invented, made-up, unreal, i

In [13]:
conversation.predict(input="And the next 4?")

' Imagined, conceptual, conceptualized, theoretical.'

In [14]:
# poet
poet_template: str = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain: LLMChain = LLMChain(
    llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template: str = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain: LLMChain = LLMChain(
    llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

In [15]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

In [16]:
# Run the poet and critic chain with a specific theme
theme: str = "the beauty of nature"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)



poem: Nature's Beauty
Romantic_expressions: False
Nature_descriptions: True


In [17]:
template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:




> Finished chain.


'\n1. Synthetic\n2. Man-made\n3. Faux\n4. Imitation\n5. Simulated\n6. Fake\n7. Replicated\n8. Manufactured\n9. Counterfeit\n10. Mock\n11. Pretend\n12. False\n13. Inauthentic\n14. Unnatural\n15. Fabricated'

In [18]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [19]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is made or produced by humans, rather than occurring naturally. It can also refer to something that is not genuine or authentic, but rather created or imitated to appear real.

Synthetic
